In [27]:
from database import *

def ticks_to_kbars(ticks, interval='1Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [28]:
import talib

def get_technical_indicator(kbars):
    
#     kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    kbars['slowk'], kbars['slowd'] = talib.STOCH(kbars.high,
                        kbars.low,
                        kbars.close,
                        fastk_period=9,
                        slowk_period=3,
                        slowk_matype=0,
                        slowd_period=3,
                        slowd_matype=0)

    upper, middle, lower = talib.BBANDS(kbars.close, 
                                        timeperiod=20, 
                                        nbdevup=2.1, 
                                        nbdevdn=2.1, 
                                        # Moving average type: simple moving average here
                                        matype=0)
    kbars['upper'] = upper
    kbars['middle'] = middle
    kbars['lower'] = lower    
    
    return kbars

In [29]:
import json

with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id = config['account'],
    passwd= config['password'],
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.


In [33]:
code = '6278'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2021-3-9')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks, '5Min')
kbars = get_technical_indicator(kbars)
kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]
# kbars = kbars[date:]

kbars

,open,close,high,low,volume,cci,macd,macdsignal,macdhist,slowk,slowd,upper,middle,lower
ts,,,,,,,,,,,,,,
2021-03-08 13:30:00,109.0,109.0,109.0,109.0,289,-127.272727,-1.163226,-1.174573,0.011347,33.333333,40.740741,111.317556,109.875,108.432444
2021-03-08 14:30:00,109.0,109.0,109.0,109.0,12,-118.309859,-1.170732,-1.173293,0.002561,0.000000,25.925926,111.163027,109.775,108.386973
2021-03-09 09:00:00,107.5,108.0,108.5,107.0,403,-330.555556,-1.429341,-1.258642,-0.170698,11.111111,14.814815,111.105277,109.625,108.144723
2021-03-09 09:05:00,108.0,108.5,108.5,107.5,117,-178.301887,-1.426574,-1.314619,-0.111954,27.777778,12.962963,110.908723,109.500,108.091277
2021-03-09 09:10:00,108.0,108.5,108.5,108.0,126,-114.323963,-1.397662,-1.342300,-0.055362,44.444444,27.777778,110.792982,109.400,108.007018
2021-03-09 09:15:00,108.0,107.0,108.0,106.5,357,-196.917148,-1.767152,-1.483917,-0.283235,38.095238,36.772487,110.900893,109.225,107.549107
2021-03-09 09:20:00,107.0,105.5,107.0,105.5,353,-213.333333,-2.380029,-1.782621,-0.597408,21.428571,34.656085,111.262820,108.975,106.687180
2021-03-09 09:25:00,105.5,106.0,106.5,105.5,196,-161.111111,-2.581837,-2.049026,-0.532810,8.465608,22.663139,111.378386,108.775,106.171614
2021-03-09 09:30:00,106.0,105.5,106.5,105.5,149,-136.111111,-2.797604,-2.298552,-0.499052,3.703704,11.199295,111.390461,108.525,105.659539


Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (700014ec1000) Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/MacBook-Pro-2.local/20764/#00020001/eULRWAq3FQ', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' I | Event: Session reconnecting
Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (700013d35000) Session '(c1,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/MacBook-Pro-2/20764/#00010001/k4mM1LQlgA', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' IP 203. | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 3) | Event: Session reconnected
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (h

In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

<SecurityType.Option: 'OPT'> fetch done.


In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:06:00
current_price = 114.0
[2021-03-02 09:06:00] sell 2456 at 114.0


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:06:00,114.0,1000,114000.0
